In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("imbikramsaha/food11")

print("Path to dataset files:", path)

100%|██████████| 519M/519M [00:05<00:00, 98.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/imbikramsaha/food11/versions/1


In [2]:
import os
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [3]:
data_dir = os.path.join(path, "food11")

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:
# Загрузка датасета
dataset = ImageFolder(root=data_dir, transform=transform)
num_classes = len(dataset.classes)

In [6]:
# Разделение на train/test
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
# Загрузка предобученной модели EfficientNet-B0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 157MB/s]


In [8]:
# Замена классификатора
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

In [9]:
# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [10]:
# Функции обучения и оценки
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    loop = tqdm(loader, desc="Training")
    for inputs, targets in loop:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    return running_loss / len(loader)

In [11]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)
    return 100 * correct / total

In [12]:
# Обучение
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_acc = evaluate(model, val_loader)
    print(f"Train Loss: {train_loss:.4f} | Val Accuracy: {val_acc:.2f}%")


Epoch 1/10


Training: 100%|██████████| 275/275 [01:22<00:00,  3.35it/s, loss=0.437]


Train Loss: 0.3523 | Val Accuracy: 90.64%

Epoch 2/10


Training: 100%|██████████| 275/275 [01:21<00:00,  3.38it/s, loss=0.246]


Train Loss: 0.2389 | Val Accuracy: 90.32%

Epoch 3/10


...

Train Loss: 0.0285 | Val Accuracy: 87.64%

Epoch 9/10


Training: 100%|██████████| 275/275 [01:21<00:00,  3.36it/s, loss=0.0116]


Train Loss: 0.0256 | Val Accuracy: 87.95%

Epoch 10/10


Training: 100%|██████████| 275/275 [01:21<00:00,  3.38it/s, loss=0.022]


Train Loss: 0.0229 | Val Accuracy: 87.18%
